In [10]:
import numpy as np
import os
import math

# load data
for file in os.listdir():
    if file[:3]=="raw":
        raw = np.load(file, allow_pickle=True)
    elif file=="cut.npy":
        cut = np.load(file, allow_pickle=True)
        
# label statistics
i, j = cut[0][0], 0                # ith time point, jth period
y0, x0, bar = [], [], np.zeros(30) # y0:which class, x0:index of the segment's end point
while i<cut[-1][-1]:               # -1:正常 0~28:剩餘時間 
    if cut[0][j]<=i<cut[1][j]:
        label = -1
    elif cut[1][j]<=i<cut[2][j]:
        label = min((cut[2][j]-i)//4, 28)
    elif cut[2][j]<=i<cut[3][j]:
        label = 0
    elif cut[3][j]<=i<cut[4][j]:
        label = -1
    else:
        if j<len(cut[0])-1:
            j+=1
            i=cut[0][j]
            continue
        else:
            break
    x0.append(i)
    y0.append(label)
    bar[int(label)]+=1
    i+=1
    
# collect data
upperBound = int(bar[1:-1].mean())*3     #-1和0都會特多 因此限制其上限為剩餘類別的3倍
totalFirst, totalLast, top = 0, 0, 0     #填入第top個資料
N = int(bar[1:-1].sum()) + upperBound*2  #-1和0這兩種
X, y = np.zeros((N,25)), np.zeros((N,30))
for i in range(len(x0)):
    if y0[i]==0:
        if totalFirst==upperBound:
            continue
        totalFirst+=1
    elif y0[i]==-1:
        if totalLast==upperBound:
            continue
        totalLast+=1
    for j in range(25):
        X[top][j] = raw[int(x0[i])-24+j]
    y[top][int(y0[i])] = 1   
    top+=1
    
# normalize, shuffle and split
np.save("mean_std.npy", [X.mean(), X.std()])
X = (X-X.mean())/X.std()
z = list(zip(X, y))
np.random.shuffle(z)
X, y = zip(*z)
X, y = np.array(X), np.array(y)

xTrain, yTrain = X[:int(N*0.6)], y[:int(N*0.6)]
xVal, yVal = X[int(N*0.6):int(N*0.8)], y[int(N*0.6):int(N*0.8)]
xTest, yTest = X[int(N*0.8):], y[int(N*0.8):]

np.save("xTrain.npy", xTrain)
np.save("xVal.npy", xVal)
np.save("xTest.npy", xTest)
np.save("yTrain.npy", yTrain)
np.save("yVal.npy", yVal)
np.save("yTest.npy", yTest)

'\nnp.save("xTrain.npy", xTrain)\nnp.save("xVal.npy", xVal)\nnp.save("xTest.npy", xTest)\nnp.save("yTrain.npy", yTrain)\nnp.save("yVal.npy", yVal)\nnp.save("yTest.npy", yTest)'

In [7]:
s = ""
for i in cut[4]:
    s+=str(int(i))+" "
s

'205 350 700 1070 1275 1645 1840 2050 2260 2465 2675 3135 3350 3550 3900 4075 '